In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

In [3]:
# get data in desirved format
dow_files_addr = "./Data/Dow_30_1_min/"
etf_files_addr = "./Data/50_ETFs_1min/"

dow_files_names = [f for f in listdir(dow_files_addr)]
etf_files_names = [f for f in listdir(etf_files_addr)]

dow_names = [f.split(".")[0] for f in dow_files_names]
etf_names = [f.split(".")[0] for f in etf_files_names]

dow_dfs = []
etf_dfs = []
headers = ["date", "time", "open_price", "close_price", "highest_price", "lowest_price", "volumn"]
dtypes={"date": "str", 
    "time":"str",
    "open_price": "float", 
    "highest_price": "float",
    "lowest_price": "float", 
    "close_price": "float",
    "volumn": "int"}
print("Start read dow file...")
for dow_name in dow_files_names:
    df = pd.read_csv(dow_files_addr + dow_name, sep=",",names=headers,dtype=dtypes, header=None)
    df["datetime"] = df["date"] + " " + df["time"]
    df["datetime"] = pd.to_datetime(df["datetime"], format="%m/%d/%Y %H:%M")
    dow_dfs.append(df)
print("Finish read dow file.")
print("---------------------")
print("Start read etf file...")
for etf_name in etf_files_names:
    df = pd.read_csv(etf_files_addr + etf_name, sep=",",names=headers,dtype=dtypes, header=None)
    df["datetime"] = df["date"] + " " + df["time"]
    df["datetime"] = pd.to_datetime(df["datetime"], format="%m/%d/%Y %H:%M")
    etf_dfs.append(df)
print("Finish read etf file.")


Start read dow file...
Finish read dow file.
---------------------
Start read etf file...
Finish read etf file.


In [23]:
# get start and end for both dow and etf that cove evey df
dow_time_slice = [min(dow_dfs[0]["datetime"]), max(dow_dfs[0]["datetime"])]
etf_time_slice = [min(etf_dfs[0]["datetime"]), max(etf_dfs[0]["datetime"])]
print("Start check dow date...")
i = 0
for df in dow_dfs:
    start = min(df["datetime"])
    if start > dow_time_slice[0]:
        print("start", dow_files_names[i])
        print(start)
        dow_time_slice[0] = start
    end = max(df["datetime"])
    if end < dow_time_slice[1]:
        print("end", dow_files_names[i])
        print(end)
        dow_time_slice[1] = end
    i += 1
print("Finish check dow date...")
print("---------------------")
print("Start check etf date...")
i = 0
for df in etf_dfs:
    start = min(df["datetime"])
    if start > etf_time_slice[0]:
        print("start", dow_files_names[i])
        print(start)
        etf_time_slice[0] = start
    end = max(df["datetime"])
    if end < etf_time_slice[1]:
        print("end", dow_files_names[i])
        print(end)
        etf_time_slice[1] = end
    i += 1
print("Finish check etf date...")

Start check dow date...
start DD.txt
2019-05-24 11:14:00
end DOW.txt
2020-06-05 18:30:00
end DWDP.txt
2019-05-31 19:36:00
end KFT.txt
2012-10-01 09:59:00
Finish check dow date...
---------------------
Start check etf date...
start AAPL.txt
2003-04-14 09:32:00
end AAPL.txt
2020-06-05 19:41:00
end AXP.txt
2020-06-05 17:38:00
start BA.txt
2008-11-19 09:30:00
start PG.txt
2009-06-25 09:40:00
Finish check etf date...


In [24]:
dow_time_slice

[Timestamp('2019-05-24 11:14:00'), Timestamp('2012-10-01 09:59:00')]

In [25]:
etf_time_slice

[Timestamp('2009-06-25 09:40:00'), Timestamp('2020-06-05 17:38:00')]

In [ ]:
tmp = dow_dfs[0]

In [37]:
tmp

,date,time,open_price,close_price,highest_price,lowest_price,volumn,datetime
0,04/27/2007,10:52,73.72,73.74,73.72,73.74,7405,2007-04-27 10:52:00
1,04/27/2007,10:53,73.74,73.80,73.74,73.80,2323,2007-04-27 10:53:00
2,04/27/2007,10:54,73.78,73.80,73.78,73.78,3340,2007-04-27 10:54:00
3,04/27/2007,10:55,73.78,73.80,73.78,73.80,2904,2007-04-27 10:55:00
4,04/27/2007,10:56,73.80,73.82,73.78,73.82,5324,2007-04-27 10:56:00
...,...,...,...,...,...,...,...,...
1470596,06/05/2020,19:44,12.05,12.05,12.05,12.05,384,2020-06-05 19:44:00
1470597,06/05/2020,19:52,12.05,12.05,12.05,12.05,416,2020-06-05 19:52:00
1470598,06/05/2020,19:55,12.04,12.04,12.04,12.04,1791,2020-06-05 19:55:00
1470599,06/05/2020,19:56,12.00,12.00,12.00,12.00,400,2020-06-05 19:56:00


In [38]:
min(tmp["datetime"])

Timestamp('2007-04-27 10:52:00')

In [39]:
max(tmp["datetime"])

Timestamp('2020-06-05 19:59:00')